In [1]:
import pandas as pd
# List of submission files and their public scores
files = [
    'Logistic_regression_submission1.csv',
    'Random_forest_classifier_submission.csv',
    'Xgb_classification_tuned2_submission.csv',
    'Shallow_nn_submission.csv',
    'DNN_submission.csv',
    'Logistic_regression_submission2.csv',
    'Lgbm_sumbission.csv'
]
scores = [0.77783, 0.75896, 0.76803, 0.76586, 0.78412, 0.77890, 0.79185]


Calculate Normalized Weights

In [2]:
# Calculate weights proportional to public scores
weights = [s / sum(scores) for s in scores]
print("Weights:", weights)


Weights: [0.14336426721714848, 0.13988627881044316, 0.14155799872823951, 0.14115803927712398, 0.14452359668605025, 0.14356148224603957, 0.145948337034955]


Load all CSVs and Merge Predictions

In [3]:
# Load all CSVs
dfs = [pd.read_csv(f) for f in files]

# Merge predictions on customer_ID
ensemble_df = dfs[0][['customer_ID']].copy()
for i, df in enumerate(dfs):
    ensemble_df[f'pred_{i}'] = df['prediction']
ensemble_df.head()


,customer_ID,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.032622,0.087185,0.016209,0.060351,0.021039,0.030283,0.037085
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.004898,0.023524,0.000375,0.009108,0.001643,0.004709,0.001281
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.041837,0.060922,0.022605,0.037364,0.013066,0.039337,0.036489
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.240636,0.391251,0.187000,0.412774,0.286931,0.236548,0.293370
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.878939,0.685837,0.945189,0.860924,0.847833,0.880827,0.838887


In [4]:
# Compute weighted average
ensemble_df['prediction'] = sum(
    weights[i] * ensemble_df[f'pred_{i}'] for i in range(len(weights))
)


In [5]:
final_submission = ensemble_df[['customer_ID', 'prediction']]
final_submission.to_csv('final_submission1.csv', index=False)
print("Final submission saved as final_submission.csv")


Final submission saved as final_submission.csv


not very good, final_submission1.csv, only get a score of 0.78750.

try weighted Top models ensemble:

In [6]:
import pandas as pd

# Load only the top 2–3 model submissions
lgbm = pd.read_csv('Lgbm_sumbission.csv')
dnn = pd.read_csv('DNN_submission.csv')
logreg = pd.read_csv('Logistic_regression_submission2.csv')

# Merge on customer_ID
df = lgbm[['customer_ID']].copy()
df['lgbm'] = lgbm['prediction']
df['dnn'] = dnn['prediction']
df['logreg'] = logreg['prediction']

# Heavily weight LGBM, then DNN, then logreg
df['prediction'] = (
    0.8 * df['lgbm'] +
    0.15 * df['dnn'] +
    0.05 * df['logreg']
)

# Save to CSV
df[['customer_ID', 'prediction']].to_csv('final_submission2.csv', index=False)
